# My turn

In [1]:
import numpy as np

In [2]:
# My particle gambling functions
def r_sample(radius):
    r = radius* ((np.random.random())**(1/3))
    return r
def alpha_sample():
    return 2*np.pi*np.random.random()
def omega_sample():
    return 2*np.random.random() - 1

def mu_sample():
    return 2*np.random.random() - 1
def gamma_sample():
    return 2*np.pi*np.random.random()
    
def x_sample(radius, alpha, omega):
    return radius*np.cos(alpha)*np.sqrt(1 - (omega**2))
def y_sample(radius, alpha, omega):
    return radius*np.sin(alpha)*np.sqrt(1 - (omega**2))
def z_sample(radius, omega):
    return radius*omega
    

In [3]:
# Alright enough lollie-gagging (I think that's how you spell it). Time to code my MC-Particle Simulation
rng = np.random.random()

class MyParticle:
    energy = 0
    
    x = 0
    y = 0
    z = 0
    
    omega_x = 0
    omega_y = 0
    omega_z = 0
    
    alive = False  
    
    def __init__(self, x, y, z, energy): 
        self.energy = energy
        
        self.x = x
        self.y = y
        self.z = z
        
        mu = mu_sample()
        gamma = gamma_sample()
        
        self.omega_x = np.sqrt(1 - mu**2)*np.cos(gamma)
        self.omega_y = np.sqrt(1 - mu**2)*np.sin(gamma)
        self.omega_z = mu
        
        self.alive = True
    
    def print_vals(self):
        print('Here are the values of this particle')
        print(f"The energy is: {self.energy}")
        print(f"The x position is: {self.x}")
        print(f"The y position is: {self.y}")
        print(f"The z position is: {self.z}") 
    
    def make_dead(self):
        self.alive = False


In [4]:
class Sphere:
    radius = 0
    E_t = 0
    
    def __init__(self, radius, E_t):
        self.radius = radius
        self.E_t = E_t
        
    
    def sample_particle(self, energy):
        r = r_sample(self.radius)
        w = 2*np.random.random() - 1
        a = 2*np.pi*np.random.random()
        
        x = x_sample(r, a, w)
        y = y_sample(r, a, w)
        z = z_sample(r, w)
        
        return MyParticle(x, y, z, energy) 
    
    # Distance determination functions -----------------

    # Distance to sphere surface
    
    def surf_dist(self, particle:MyParticle):
        b = 2*(particle.x*particle.omega_x + particle.y*particle.omega_y + particle.z*particle.omega_z)
        c = particle.x**2 + particle.y**2 + particle.z**2 - self.radius**2
        
        pos = (-b + np.sqrt(b**2 - 4*c))*0.5
        if pos > 0:
            return pos
            
        else:
            return (-b - np.sqrt(b**2 - 4*c))*0.5
        
    # Distance to next collision assuming
    def coll_dist_abs(self):
        return -np.log(np.random.random())/self.E_t  
    
    def is_in_sphere(self, particle:MyParticle, x = 0, y = 0, z = 0):
     if (x - particle.x)**2 + (y - particle.y)**2 + (z - particle.z)**2 <= self.radius**2:
        return True
     else:
         return False
    
    

In [5]:
def history(histories, energy, r, E_t):
    leaked = 0
    sphere = Sphere(r, E_t)
    
    particle_bank = np.array([sphere.sample_particle(energy) for _ in range(int(histories))], dtype=object)
    bank_tracker = histories
    i = 0
    
    while i < bank_tracker:
        
        while particle_bank[i].alive == True:
            l_s = sphere.surf_dist(particle_bank[i])
            l_c = sphere.coll_dist_abs()
            
            if l_s < l_c:
                particle_bank[i].make_dead()
                leaked += 1
            else:
                particle_bank[i].make_dead()
        
        i += 1
        
    
    return leaked



In [6]:
# Lets generate a monoenergetic sample of particles
# I am thinking 1E6 neutrons with an energy of 1 MeV scattered about a sphere of radius 5 units
mono_energy = 4.5 # MeV
sphere_radius = 5
E_a = 0.47
histories = 1E7

leaked = history(histories, mono_energy, sphere_radius, E_a)
    
print(f"In the purely absorbing sphere of radius {sphere_radius} and E_a of {E_a}, \n {leaked} particles were leaked out of {histories}")
print(f"Leakage rate is {leaked/histories}")

In the purely absorbing sphere of radius 5 and E_a of 0.47, 
 2917341 particles were leaked out of 10000000.0
Leakage rate is 0.2917341


In [7]:
# Check
P_0 = (0.375)*((E_a*sphere_radius)**(-3)) *(2*(E_a*sphere_radius)**2 - 1 + (1+2*E_a*sphere_radius)*np.exp(-2*E_a*sphere_radius))
print(f"{P_0}")

0.29175162765898693
